In [1]:
import numpy as np


class Data:
    def __init__(self, name, batch_size):  # 数据所在的文件名name和batch中图片的数量batch_size
        with open(name, 'rb') as f:
            data = np.load(f, allow_pickle=True)
        self.x = data[0]  # 输入x
        self.y = data[1]  # 预期正确输出y
        self.l = len(self.x)
        self.batch_size = batch_size
        self.pos = 0  # pos用来记录数据读取的位置

    def forward(self):
        pos = self.pos
        bat = self.batch_size
        l = self.l
        if pos + bat >= l:  # 已经是最后一个batch时，返回剩余的数据，并设置pos为开始位置0
            ret = (self.x[pos:l], self.y[pos:l])
            self.pos = 0
            #print(l)
            index = list(range(l))
            np.random.shuffle(index)  # 将训练数据打乱
            self.x = self.x[index]
            self.y = self.y[index]
        else:  # 不是最后一个batch, pos直接加上batch_size
            ret = (self.x[pos:pos + bat], self.y[pos:pos + bat])
            self.pos += self.batch_size

        return ret, self.pos  # 返回的pos为0时代表一个epoch已经结束

    def backward(self, d):  # 数据层无backward操作
        pass


class FullyConnect:
    def __init__(self, l_x, l_y):  # 两个参数分别为输入层的长度和输出层的长度
        # 使用随机数初始化参数，请暂时忽略这里为什么多了np.sqrt(l_x)
        self.weights = np.random.randn(l_y, l_x) / np.sqrt(l_x)
        self.bias = np.random.randn(l_y, 1)  # 使用随机数初始化参数
        self.lr = 0  # 先将学习速率初始化为0，最后统一设置学习速率

    def forward(self, x):
        self.x = x  # 把中间结果保存下来，以备反向传播时使用
        self.y = np.array([np.dot(self.weights, xx) +
                           self.bias for xx in x])  # 计算全连接层的输出
        return self.y  # 将这一层计算的结果向前传递

    def backward(self, d):
        # 根据链式法则，将反向传递回来的导数值乘以x，得到对参数的梯度
        ddw = [np.dot(dd, xx.T) for dd, xx in zip(d, self.x)]
        self.dw = np.sum(ddw, axis=0) / self.x.shape[0]
        self.db = np.sum(d, axis=0) / self.x.shape[0]
        self.dx = np.array([np.dot(self.weights.T, dd) for dd in d])

        # 更新参数
        self.weights -= self.lr * self.dw
        self.bias -= self.lr * self.db
        return self.dx  # 反向传播梯度


class Sigmoid:
    def __init__(self):  # 无参数，不需初始化
        pass

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, x):
        self.x = x
        self.y = self.sigmoid(x)
        return self.y

    def backward(self, d):
        sig = self.sigmoid(self.x)
        self.dx = d * sig * (1 - sig)
        return self.dx  # 反向传递梯度


class QuadraticLoss:
    def __init__(self):
        pass

    def forward(self, x, label):
        self.x = x
        # 由于我们的label本身只包含一个数字，我们需要将其转换成和模型输出值尺寸相匹配的向量形式
        self.label = np.zeros_like(x)
        for a, b in zip(self.label, label):
            a[b] = 1.0  # 只有正确标签所代表的位置概率为1，其他为0
        self.loss = np.sum(np.square(x - self.label)) / \
            self.x.shape[0] / 2  # 求平均后再除以2是为了表示方便
        return self.loss

    def backward(self):
        self.dx = (self.x - self.label) / self.x.shape[0]  # 2被抵消掉了
        return self.dx


class CrossEntropyLoss:
    def __init__(self):
        pass

    def forward(self, x, label):
        self.x = x
        self.label = np.zeros_like(x)
        for a, b in zip(self.label, label):
            a[b] = 1.0
        # np.nan_to_num()避免log(0)得到负无穷的情况
        self.loss = np.nan_to_num(-self.label *
                                  np.log(x) - ((1 - self.label) * np.log(1 - x)))
        self.loss = np.sum(self.loss) / x.shape[0]
        return self.loss

    def backward(self):
        self.dx = (self.x - self.label) / self.x / \
            (1 - self.x)  # 分母会与Sigmoid层中的对应部分抵消
        return self.dx


class Accuracy:
    def __init__(self):
        pass

    def forward(self, x, label):  # 只需forward
        self.accuracy = np.sum(
            [np.argmax(xx) == ll for xx, ll in zip(x, label)])  # 对预测正确的实例数求和
        self.accuracy = 1.0 * self.accuracy / x.shape[0]
        return self.accuracy

In [2]:
def main():
    datalayer1 = Data('data/train.npy', 1024)  # 用于训练，batch_size设置为1024
    # 用于验证，所以设置batch_size为10000,一次性计算所有的样例
    datalayer2 = Data('data/validate.npy', 10000)
    inner_layers = []
    inner_layers.append(FullyConnect(17 * 17, 20))
    inner_layers.append(Sigmoid())
    inner_layers.append(FullyConnect(20, 26))  # 增加一个隐层
    inner_layers.append(Sigmoid())
    losslayer = QuadraticLoss()
    accuracy = Accuracy()

    for layer in inner_layers:
        layer.lr = 1000.0  # 为所有中间层设置学习速率

    epochs = 20
    for i in range(epochs):
        print('epochs:', i)
        losssum = 0
        iters = 0
        while True:
            data, pos = datalayer1.forward()  # 从数据层取出数据
            x, label = data
            for layer in inner_layers:  # 前向计算
                x = layer.forward(x)

            loss = losslayer.forward(x, label)  # 调用损失层forward函数计算损失函数值
            losssum += loss
            iters += 1
            d = losslayer.backward()  # 调用损失层backward函数曾计算将要反向传播的梯度

            for layer in inner_layers[::-1]:  # 反向传播
                d = layer.backward(d)

            if pos == 0:  # 一个epoch完成后进行准确率测试
                data, _ = datalayer2.forward()
                x, label = data
                for layer in inner_layers:
                    x = layer.forward(x)
                accu = accuracy.forward(x, label)  # 调用准确率层forward()函数求出准确率
                print('loss:', losssum / iters)
                print('accuracy:', accu)
                break


if __name__ == '__main__':
    main()

epochs: 0
loss: 0.630214458669289
accuracy: 0.0412
epochs: 1
loss: 0.48027311132686334
accuracy: 0.0662
epochs: 2
loss: 0.4782947706358594
accuracy: 0.0405
epochs: 3
loss: 0.47534284719385606
accuracy: 0.0691
epochs: 4
loss: 0.471010362060996
accuracy: 0.0412
epochs: 5
loss: 0.46625704321856604
accuracy: 0.1077
epochs: 6
loss: 0.4593167713345355
accuracy: 0.1194
epochs: 7
loss: 0.4521781406223829
accuracy: 0.1866
epochs: 8
loss: 0.4415712056666338
accuracy: 0.1204
epochs: 9
loss: 0.4350049276649647
accuracy: 0.3275
epochs: 10
loss: 0.4213039423067082
accuracy: 0.2385
epochs: 11
loss: 0.4109893612684813
accuracy: 0.3475
epochs: 12
loss: 0.3949866039069355
accuracy: 0.4455
epochs: 13
loss: 0.37457157982689615
accuracy: 0.3949
epochs: 14
loss: 0.3542813991536828
accuracy: 0.4105
epochs: 15
loss: 0.3325247009155853
accuracy: 0.392
epochs: 16
loss: 0.31118838012904754
accuracy: 0.6353
epochs: 17
loss: 0.2827904923773702
accuracy: 0.5971
epochs: 18
loss: 0.2599848604993026
accuracy: 0.6999
e

In [3]:
def main():
    datalayer1 = Data('data/train.npy', 1024)  # 用于训练，batch_size设置为1024
    # 用于验证，所以设置batch_size为10000,一次性计算所有的样例
    datalayer2 = Data('data/validate.npy', 10000)
    inner_layers = []
    inner_layers.append(FullyConnect(17 * 17, 20))
    inner_layers.append(Sigmoid())
    inner_layers.append(FullyConnect(20, 26))
    inner_layers.append(Sigmoid())
    losslayer = CrossEntropyLoss()
    accuracy = Accuracy()

    for layer in inner_layers:
        layer.lr = 1.0  # 为所有中间层设置学习速率

    epochs = 20
    for i in range(epochs):
        print('epochs:', i)
        losssum = 0
        iters = 0
        while True:
            data, pos = datalayer1.forward()  # 从数据层取出数据
            x, label = data
            for layer in inner_layers:  # 前向计算
                x = layer.forward(x)

            loss = losslayer.forward(x, label)  # 调用损失层forward函数计算损失函数值
            losssum += loss
            iters += 1
            d = losslayer.backward()  # 调用损失层backward函数曾计算将要反向传播的梯度

            for layer in inner_layers[::-1]:  # 反向传播
                d = layer.backward(d)

            if pos == 0:  # 一个epoch完成后进行准确率测试
                data, _ = datalayer2.forward()
                x, label = data
                for layer in inner_layers:
                    x = layer.forward(x)
                accu = accuracy.forward(x, label)  # 调用准确率层forward()函数求出准确率
                print('loss:', losssum / iters)
                print('accuracy:', accu)
                break


if __name__ == '__main__':
    main()

epochs: 0
loss: 4.089140659629275
accuracy: 0.6332
epochs: 1
loss: 2.622252312546044
accuracy: 0.8384
epochs: 2
loss: 1.9187565336590375
accuracy: 0.8608
epochs: 3
loss: 1.4983254214685229
accuracy: 0.9065
epochs: 4
loss: 1.2147071766821589
accuracy: 0.9462
epochs: 5
loss: 1.0107972006645318
accuracy: 0.9477
epochs: 6
loss: 0.8556984805185829
accuracy: 0.9591
epochs: 7
loss: 0.7396769749037955
accuracy: 0.9609
epochs: 8
loss: 0.6482363670047515
accuracy: 0.9645
epochs: 9
loss: 0.575212806737187
accuracy: 0.9686
epochs: 10
loss: 0.5243189456596665
accuracy: 0.9712
epochs: 11
loss: 0.4752971367790839
accuracy: 0.9733
epochs: 12
loss: 0.43867731204574173
accuracy: 0.9744
epochs: 13
loss: 0.4072197303314605
accuracy: 0.9747
epochs: 14
loss: 0.3808470715239229
accuracy: 0.9762
epochs: 15
loss: 0.36218208623563597
accuracy: 0.9774
epochs: 16
loss: 0.3373145000418363
accuracy: 0.9776
epochs: 17
loss: 0.3215880140523324
accuracy: 0.9776
epochs: 18
loss: 0.304686626717045
accuracy: 0.9783
epoch